### Importanto as bibliotecas

In [5]:
import re #biblioteca para realizar buscas dentro do arquivo texto
import os
import copy

### Definindo uma classe automato e criando uma função que leia o automato dado um arquivo .txt

In [6]:
class automato:        
    def __init__(self, arqtxt): #recebe como parametro um arquivo texto contendo o automato
        
        #Retorna os simbolos do alfabeto, os estados do automato, o estado inicial e os estados finais
        quadrupla = re.search("\(.*?\)",arqtxt) #Procura a parte da string entre parenteses e a retorna (com os parenteses) dentro de um objeto match
        quadrupla = quadrupla.group(0) #Extrai a string do objeto match
        quadrupla = quadrupla[1:len(quadrupla)-1] #Retira os parenteses do começo e do final

        #filtra os estados
        simbolos = re.search("\{.*?\}", quadrupla)
        simbolos = simbolos.group(0)
        simbolos = simbolos[1:len(simbolos)-1]
        self.simbolos = simbolos.split(',')  #Salva os simbolos do objeto automato

        tripla = quadrupla[len(simbolos)+3:] #passa para a variavel tripla a variavel quadrupla menos os estados

        #filtra os simbolos
        estados = re.search("\{.*?\}", tripla)
        estados = estados.group(0)
        estados = estados[1:len(estados)-1]
        self.estados = estados.split(',') #Salva os estados do objeto automato

        dupla = tripla[len(estados)+3:] #passa para a variavel dupla a variavel tripla menos os simbolos

        #filtra o estado inicial
        restante = dupla.split(',')
        self.inicial = restante[0]
        if(len(restante) == 2):
            self.finais = restante[1]
        else:        
            #filtra os estados finais
            finais = dupla[len(self.inicial)+1:]
            finais = finais[1:len(finais)-1]
            self.finais = finais.split(',')
            
        #Salva a tabela de transição como uma lista de listas
        #Da um slice em dados a partir do index inicial da palavra Prog + o tamanho da palavra prog + 1 (pois tem um '\n' dps de prog)
        transitiv_raw = (arqtxt[arqtxt.index('Prog') + len('Prog') + 1:])
        transitiv_raw = transitiv_raw.split('\n') #salva cada elemento em uma linha

        transitiv = [] #inicializa a lista vazia que vai armazenar a tabela de transitividade
        for x in transitiv_raw: #itera por cada elemento da tabela de transitividade
            inst = [] #inicializa a lista que vai salvar os 3 elemento de cada transição
            x = x.split('=') #divide o estado inicial + entrada do estado resultante
    
            atual = x[0][1:len(x[0])-1] #salva na variavel atual o estado atual + a variavel de entrada sem os parenteses
            atual = atual.split(',') #faz com que atual contenha uma lista contendo o estado atual e a variavel
            inst.append(atual[0]) #salva o estado atual
            inst.append(atual[1]) #salva a variavel da transição
            inst.append(x[1]) #salva o estado resultante no terceiro espaço da lista
        
            transitiv.append(inst) #salva a lista interna na lista externa
        self.transitiv = transitiv
        
class automatoP:
    def __init__(self,automato):
        self.estados = []
        self.simbolos = []
        self.inicial  = automato.inicial
        self.finais = []
        self.transitiv = [] 

# Funcao de conversão

In [7]:
def converte(arquivo):
    openfile = open(arquivo,"r", encoding="utf-8",) #Lembrar de incluir o encoding para que ele leia os acentos corretamente
    dados = openfile.read()
    
    automatoND = automato(dados) #le o arquivo e cria um automato nao deterministico
    automatoDFA = automatoP(automatoND) #instancia o automato deterministico
    
    visited = [] #estados  já visitados
    
    recursao([automatoDFA.inicial], automatoND.simbolos, automatoND.transitiv, visited, automatoDFA) #preenche o automato deterministico
    
    # agora temos que transformar as listas de estados em um estado só representado pela concatenação dos itens da lista:    
    # primeiro na lista de estados:
    counter = 0
    for estado in automatoDFA.estados:
        if type(estado) == list:   # se for um estado ou mais em forma de lista, converte
            string = ""
            for q in estado:   # para cada estado dentro desse estado que deveria ser único, concatena-os
                string = string+q
            automatoDFA.estados[counter] = string
        counter +=1

    
    # depois fazemos o mesmo para as transições:
    counter = 0
    for trans in automatoDFA.transitiv:
        # trata estado atual da transição
        if type(trans[0]) == list:   # se for um estado ou mais em forma de lista, converte
            string = ""
            for q in trans[0]:   # para cada estado dentro desse estado que deveria ser único, concatena-os
                string = string+q
            automatoDFA.transitiv[counter][0] = string
        
        # trata estado final da transição
        if type(trans[2]) == list:   # se for um estado ou mais em forma de lista, converte
            string = ""
            for q in trans[2]:   # para cada estado dentro desse estado que deveria ser único, concatena-os
                string = string+q
            automatoDFA.transitiv[counter][2] = string
        counter += 1

    # os estados finais serão todos aqueles que tiverem o próprio final do NFA ou algum estados agrupado com o final:
    for estado in automatoDFA.estados:
        # aqui só faz um teste para saber se existe apenas um final, neste caso deve-se passar uma lista desse um elemento
        finais = automatoND.finais
        if type(finais) == str:
            finais = [finais]
    
        for efinal in finais:
            if (efinal in estado) and (not(estado in automatoDFA.finais)):
                automatoDFA.finais.append(estado)

    return automatoDFA

### Teste da funcao de conversão

In [8]:
teste = converte('automatop.txt')


print("\n\nestados:", teste.estados)
print("simbolos:", teste.simbolos)
print("Estado inicial:", teste.inicial)
print("Estados finais:", teste.finais)

print("\nTabela de transição:")
for x in teste.transitiv:
    print(x)

NameError: name 'recursao' is not defined

# Funcao de aceitação

In [159]:
def checaSeAceita(automato,palavra):
    
    # começa no estado inicial sempre
    eAtual = automato.inicial
    
    # vai lendo os simbolos da palavra e tentando percorrer o autômato
    for simbolo in palavra:
        transValida = False   # se encontrar uma transição válida, muda
        # procura uma transição válida na lista de transições
        for trans in automato.transitiv:
            # se o simbolo e a palavra são uma transição válida, atualiza o estado atual e passa para o próximo símbolo
            if (eAtual == trans[0]) and (simbolo == trans[1]):
                eAtual = trans[2]
                transValida = True
                break
        
        if transValida:
            continue   # vai para o próximo símbolo
        
        # se não encontrar, retorna com "REJEITA"
        return "REJEITA"
    
    
    # se finalizou-se o loop sem retornar "REJEITA", testa se o estado atual é um estado final
    if eAtual in automato.finais:
        return "ACEITA"
    
    else:
        return "REJEITA"   
    
def reconhecimento(automato):
    listaDePalavras = []

    print('------- Teste do autômato para reconhecimento ou não de palavras  -------')
    print('Digite uma palavra separando os símbolos por vírgula, sem espaço, para verificar seu reconhecimento:')
    listaDePalavras.append(input())

    while True:
        print('Deseja fornecer mais palavras? s/n')
        select = input()
    
        if select == "s":
            print('Digite uma palavra separando os símbolos por vírgula, sem espaço, para verificar seu reconhecimento:')
            listaDePalavras.append(input())
        else:
            break

    print("\n")
    counter = 1
    for palavra in listaDePalavras:
        palavra = palavra.split(",")
        print("Palavra "+ str(counter) + ": ", checaSeAceita(automato,palavra))
        counter += 1

# Teste final (combinando as duas funções)

In [163]:
teste = converte('automatop.txt')
reconhecimento(teste)

------- Teste do autômato para reconhecimento ou não de palavras  -------
Digite uma palavra separando os símbolos por vírgula, sem espaço, para verificar seu reconhecimento:
Liga,Ficha Inserida,Alavanca,Diamante,Diamante,Diamante,Jackpot,Retira Prêmio,Libera Inserção de Ficha,Desliga
Deseja fornecer mais palavras? s/n
s
Digite uma palavra separando os símbolos por vírgula, sem espaço, para verificar seu reconhecimento:
Liga,Liga
Deseja fornecer mais palavras? s/n
s
Digite uma palavra separando os símbolos por vírgula, sem espaço, para verificar seu reconhecimento:
Liga,Desliga
Deseja fornecer mais palavras? s/n
n


Palavra 1:  ACEITA
Palavra 2:  REJEITA
Palavra 3:  ACEITA


### Exemplos de palavras que sao aceitas no arquivo automatop.txt:
1. Liga,Ficha Inserida,Alavanca,7,Limão,Exibe Mensagem de Derrota,Libera Inserção de Ficha,Desliga
2. Liga,Ficha Inserida,Alavanca,Diamante,Diamante,Diamante,Jackpot,Retira Prêmio,Libera Inserção de Ficha,Desliga
3. Liga,Ficha Inserida,Alavanca,Diamante,Diamante,7,Exibe Mensagem de Derrota,Libera Inserção de Ficha,Ficha Inserida,Alavanca,7,7,Limão,Exibe Mensagem de Derrota,Libera Inserção de Ficha,Desliga
4. Liga,Ficha Inserida,Alavanca,Limão,Limão,Limão,Jackpot,Retira Prêmio,Libera Inserção de Ficha,Ficha Inserida,Alavanca,7,7,7,Jackpot,Retira Prêmio,Libera Inserção de Ficha,Desliga
5. Liga,Ficha Inserida,Alavanca,7,7,Cereja,Exibe Mensagem de Derrota,Libera Inserção de Ficha,Desliga

### Exemplos de palavras que sao rejeitadas no arquivo automatop.txt:
1. Liga,Retira Prêmio
2. Libera Inserção De Ficha,Limão,Limão,7,7,Diamante,Alavanca
3. Liga,Ficha Inserida,Retira Prêmio,Desliga
4. Liga,Alavanca,Cereja,Cereja,Cereja,Jackpot,Retira Prêmio,Libera Inserção de Ficha,Desliga
5. Liga,Liga